In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
sys.path.append('..')

In [26]:
import altair as alt
import pandas as pd

name_mapper = {
    "random": "Random",
    "scattershot": "ScatterShot"
}

def visualize_trajectory(scores, key):
    source = []
    # process the data
    for condition in ["random", "scattershot"]:
        for scores_per_simulate in scores:
            for score in scores_per_simulate[condition]:
                data = {
                    "N(in-context examples)": score["n"],
                    "Condition": name_mapper[condition]
                }
                data[key] = score["score"][key]
                source.append(data)
    df = pd.DataFrame(source)
    
    # line
    line = alt.Chart(df).mark_line(strokeWidth=2).encode(
        x=alt.X("N(in-context examples):Q"),
        y=alt.Y(f"mean({key}):Q", scale=alt.Scale(zero=False), axis=alt.Axis(format='%')),
        color=alt.Color("Condition:N", legend=alt.Legend(orient="right"))
    )
    
    # create the dots
    dot = alt.Chart(df).mark_circle(size=100).encode(
        x=alt.X("N(in-context examples):Q"),
        y=alt.Y(f"mean({key}):Q", scale=alt.Scale(zero=False)),
        color=alt.Color("Condition:N")
    )
    
    # create error bar
    
    error_bars = alt.Chart(df).mark_errorbar(extent='stdev').encode(
        x=alt.X("N(in-context examples):Q"),
        y=alt.Y(f"{key}:Q", scale=alt.Scale(zero=False)),
        color=alt.Color("Condition:N")
    )

    
    chart = line + dot + error_bars
    chart = chart.properties(width=190, height=125)
    return chart

def visualize_trajectories(results, key):
    charts = [visualize_trajectory([r], key).properties(title=r["iteration"]) for r in results]
    return alt.hconcat(*charts).resolve_scale(y='shared')


# TEMP

In [91]:
## TODO: change the distance to 0.5
## MAX_EXAMPLE_COUNT = 40

# get the text data
import json
import numpy as np
with open("data/timeml_data/dataset.json", "r") as f:
    dataset = json.load(f)

from main.exps.simulation import simulate_train
from main.exps.evals import evaluate_timeml
description = "Extract all the mentioned dates as detailed as possible, in the ISO format of YEAR-MONTH-DAY."

temp_checkpoints, temp_testcases, temp_scores = simulate_train(
    dataset, description, evaluate_timeml,
    nsimulation=1, 
    ntest=0,
    verbose=True,
    #testcases=testcases, # if you set this then the test case is your preset case
    random_state=42, # default to None, if you set then the test case is always selected to be the same, but it's not your input.
    is_evaluate_checkpoint=False,
    is_suggest_pred=False,
    is_select_by_pred_certainty=False,
    is_select_by_diversity=True,
    is_skip_random=False, 
    llm="gpt3",
    text_preprocess=lambda x: x.split("] ")[-1])

Simulation round: 0
Setup sentence transformer.
Iteration: 0
Core templates: [DATE]
Number of clusters: 13
--- CLUSTER ----
cluster size: 67 | examined:  1 | succeeded:  0
accuracy: 0.0
sample_term: 1.048147073968205
reward: 4.204692619390966
mu: 5.252839693359171
[Posted: 2013-03-22] she thought her husband devised the plan after he was fired from his job in July. [July]
[Posted: 2013-03-22] State police arrested both men in early August on charges of insurance fraud, conspiracy and filing a false report. [early August]
[Posted: 2013-03-22] Evana Roth told CNN in August [August]
[Posted: 2013-03-22] Raymond Roth's attorney, Brian Davis, denied in August that Roth had involved his son in the scheme. [August]
[Posted: 2013-03-22] A four-week, phased closure of the 149 control towers will begin on April 7, the FAA said. [April 7, four-week]
[Posted: 2013-03-18] Microsoft said it has identified three companies for the China program to run through June. [June]
[Posted: 1989-10-30] Waxman I

0it [00:00, ?it/s]

Scattershot checkpoint: 43
3dc31b2f44694e9ca11e113ca1a12105
Extract all the mentioned dates as detailed as possible, in the ISO format of YEAR-MONTH-DAY.
| Positive Examples (35): 
	('[Posted: 2015-02-16] Understanding Green Labelling (Leed and Green Mark), 27 February 2015.', '27 February 2015 == 2015-02-27', 'Eligible')
	("[Posted: 1998-02-27] Seventy-five million copies of the rifle have been built since it entered production in February 1947, making it history's most widely distributed weapon.", 'February 1947 == 1947-02', 'Eligible')
	("[Posted: 1989-10-26] Mr. Schwarz also said costs associated with U.S. Trust's planned move to midtown Manhattan from Wall Street will continue to be a drag on earnings through 1990.", '1990 == 1990', 'Eligible')
	('[Posted: 2015-04-01] Your center of gravity is strong enough today to withstand a c...', 'today == 2015-04-01', 'Eligible')
	('[Posted: 2014-06-13] Might bump into you during my reservist in September?', 'September == 2014-09', 'Eligible

Iteration: 0
 --- [Random] selected input: ----
[Posted: 2000-01-05] ``But that's his home.'' []
[Posted: 2009-12-24] Walking down the most Christmassy street here today with my cousins!! [today]
[Posted: 1989-10-27] He wouldn't specify what it was. []
[Posted: 2014-10-01] The Singapore Environment Council will be participating in the 5th... []
[Posted: 2013-03-22] Her attorney, Lenard Leeds, said she had been unaware of the ruse before she uncovered the e-mail correspondence. []
[Posted: 2000-04-01] As a father and the grandfather of eight children -- including a boy who just turned 6, the same age as Elian -- Castro said he sympathized with the little boy's father, Juan Miguel Gonzalez. [6]
[Posted: 1989-10-30] Mr. Junius said Nashua's "intention is to remain an independent public company." []
[Posted: 2015-02-12] Communication today was tip top guys!!!!!! [today]
[Posted: 2014-03-22] Marmite cheese toast, eggs, banana chocolate tart, #coffee. []
[Posted: 2013-08-28] Can you call me 

0it [00:00, ?it/s]

Baseline checkpoint: 43
2aa0d90356654e379e108a669badfd03
Extract all the mentioned dates as detailed as possible, in the ISO format of YEAR-MONTH-DAY.
| Positive Examples (25): 
	('[Posted: 2015-02-16] Understanding Green Labelling (Leed and Green Mark), 27 February 2015.', '27 February 2015 == 2015-02-27', 'Eligible')
	("[Posted: 1998-02-27] Seventy-five million copies of the rifle have been built since it entered production in February 1947, making it history's most widely distributed weapon.", 'February 1947 == 1947-02', 'Eligible')
	("[Posted: 1989-10-26] Mr. Schwarz also said costs associated with U.S. Trust's planned move to midtown Manhattan from Wall Street will continue to be a drag on earnings through 1990.", '1990 == 1990', 'Eligible')
	('[Posted: 2009-12-24] Walking down the most Christmassy street here today with my cousins!!', 'today == 2009-12-24', 'Eligible')
	('[Posted: 2013-03-22] State police arrested both men in early August on charges of insurance fraud, conspiracy

In [78]:
from main.exps.simulation import save_simulation_results

save_simulation_results(temp_checkpoints, temp_testcases, temp_scores, "simulations/temp/iter7")

In [7]:
import json
import os
user_results = []

result_folder = "/home/wtshuang/sourcetree/interactive-perturb-fn/notebooks/simulations/temp/"
iterations = [5,6,7]
results = []
for idx, i in enumerate(iterations):
    filename = f"{result_folder}iter{i}/scores.json"
    with open(filename, "r") as f:
        d = json.load(f)
        d[0]["iteration"] = f"Temporal-{idx+1}"
        results += d
identify_f = visualize_trajectories(results, "identify-F")
identify_f

alt.HConcatChart(...)

In [5]:
visualize_trajectories(results, "value-F")

alt.HConcatChart(...)

# QA

In [ ]:
import json
import numpy as np
with open("data/vqa_impls/dataset_logeq_relabel.json", "r") as f:
    dataset = json.load(f)
    
from main.exps.simulation import simulate_train
from main.exps.evals import compute_em_rouge
description = "Rewrite the following question-answer pair to an logically equivalent alternative pair."

qa_checkpoints, qa_testcases, qa_scores = simulate_train(
    dataset, description, compute_em_rouge, 
    nsimulation=1, 
    #testcases=testcases, # if you set this then the test case is your preset case
    random_state=42, # default to None, if you set then the test case is always selected to be the same, but it's not your input.
    ntest=100,
    verbose=True,
    is_evaluate_checkpoint=False,
    is_suggest_pred=False,
    is_select_by_pred_certainty=False,
    is_select_by_diversity=True,
    is_skip_random=False,
    llm="gpt3")

In [64]:
from main.exps.simulation import save_simulation_results

save_simulation_results(qa_checkpoints, qa_testcases, qa_scores, "simulations/qa/iter7")




# visualization

In [27]:
import json
import os
user_results = []

result_folder = "/home/wtshuang/sourcetree/interactive-perturb-fn/notebooks/simulations/temp/"
iterations = [5,6,7]
results = []
for idx, i in enumerate(iterations):
    filename = f"{result_folder}iter{i}/scores.json"
    with open(filename, "r") as f:
        d = json.load(f)
        d[0]["iteration"] = f"Temporal-{idx+1}"
        results += d
identify_f = visualize_trajectories(results, "identify-F")
identify_f

alt.HConcatChart(...)

In [28]:
import json
import os
user_results = []

result_folder = "/home/wtshuang/sourcetree/interactive-perturb-fn/notebooks/simulations/qa/"
iterations = [1,6,7]
results = []
for idx, i in enumerate(iterations):
    filename = f"{result_folder}iter{i}/scores.json"
    with open(filename, "r") as f:
        d = json.load(f)
        d[0]["iteration"] = f"QA-pair-{idx+1}"
        results += d
rouge =visualize_trajectories(results, "rouge")

rouge

alt.HConcatChart(...)

In [25]:
chart = alt.vconcat(identify_f, rouge).resolve_scale(color='shared', y="shared")
chart

alt.VConcatChart(...)

In [24]:
chart.save("simulate_traj.pdf")

WARN Infinite extent for field "lower_identify-F": [Infinity, -Infinity]
WARN Infinite extent for field "upper_identify-F": [Infinity, -Infinity]
WARN Infinite extent for field "lower_identify-F": [Infinity, -Infinity]
WARN Infinite extent for field "upper_identify-F": [Infinity, -Infinity]
WARN Infinite extent for field "lower_identify-F": [Infinity, -Infinity]
WARN Infinite extent for field "upper_identify-F": [Infinity, -Infinity]
WARN Infinite extent for field "N(in-context examples)": [Infinity, -Infinity]
WARN Infinite extent for field "N(in-context examples)": [Infinity, -Infinity]
WARN Infinite extent for field "N(in-context examples)": [Infinity, -Infinity]
WARN Infinite extent for field "lower_rouge": [Infinity, -Infinity]
WARN Infinite extent for field "upper_rouge": [Infinity, -Infinity]
WARN Infinite extent for field "lower_rouge": [Infinity, -Infinity]
WARN Infinite extent for field "upper_rouge": [Infinity, -Infinity]
WARN Infinite extent for field "lower_rouge": [Infini